### Problem Description

#### Dataset: 
Training and Test Dataset is attached in the mail.

#### Problem Description:
Concrete is the most important material in civil engineering. The concrete compressive strength is a highly nonlinear function of age and ingredients. “Concrete compressive strength” is the dependent variable, which needs to be predicted based on the below independent variables.

#### Independent Variables:
* Cement (component 1) -- kg in a m3 mixture 
* Blast Furnace Slag (component 2) -- kg in a m3 mixture 
* Fly Ash (component 3) - kg in a m3 mixture
* Water (component 4) -- kg in a m3 mixture 
* Superplasticizer (component 5) -- kg in a m3 mixture 
* Coarse Aggregate (component 6) -- kg in a m3 mixture 
* Fine Aggregate (component 7) -- kg in a m3 mixture 
* Age -- Day (1~365) 

#### Dependent Variable:
* Concrete compressive strength -- MPa

In [0]:
# ref:
# https://docs.databricks.com/data/data-sources/azure/azure-storage.html#mount-azure-blob
# https://www.youtube.com/watch?v=zwMksSEjNvU

try:
  dbutils.fs.unmount('/mnt/hackathon')
except:
  pass

dbutils.fs.mount(
  source = "wasbs://hackathon@synapseadlsak.blob.core.windows.net",
  mount_point = "/mnt/hackathon",
  # use SAS of the container or ACCESS KEY of the storage account
  # use READ, WRITE, DELETE, and LIST as permissions for SAS to enable spark to read and write into and from BLOB storage.
  extra_configs =  {"fs.azure.sas.hackathon.synapseadlsak.blob.core.windows.net":
                    "sp=rwdl&st=2021-06-09T18:47:57Z&se=2021-06-24T02:47:57Z&spr=https&sv=2020-02-10&sr=c&sig=HBsJz5SwjmwS4gp3BHYPS737IcBLb9irS78Cr7Gs54U%3D"})

/mnt/hackathon has been unmounted.
Out[1]: True

In [0]:
%fs ls mnt/hackathon

path,name,size
dbfs:/mnt/hackathon/Building_Strength_Test.csv,Building_Strength_Test.csv,9603
dbfs:/mnt/hackathon/Building_Strength_Train.csv,Building_Strength_Train.csv,30500


In [0]:
train_df = spark.read.csv('/mnt/hackathon/Building_Strength_Train.csv', header=True, inferSchema=True)
test_df = spark.read.csv('/mnt/hackathon/Building_Strength_Test.csv', header=True, inferSchema=True)

In [0]:
%fs ls /mnt/hackathon

path,name,size
dbfs:/mnt/hackathon/Building_Strength_Test.csv,Building_Strength_Test.csv,9603
dbfs:/mnt/hackathon/Building_Strength_Train.csv,Building_Strength_Train.csv,30500


In [0]:
train_df.printSchema()
test_df.printSchema()

root
-- cement: double (nullable = true)
-- slag: double (nullable = true)
-- flyash: double (nullable = true)
-- water: double (nullable = true)
-- superplasticizer: double (nullable = true)
-- coarseaggregate: double (nullable = true)
-- fineaggregate: double (nullable = true)
-- age: integer (nullable = true)
-- csMPa: double (nullable = true)

root
-- cement: double (nullable = true)
-- slag: double (nullable = true)
-- flyash: double (nullable = true)
-- water: double (nullable = true)
-- superplasticizer: double (nullable = true)
-- coarseaggregate: double (nullable = true)
-- fineaggregate: double (nullable = true)
-- age: integer (nullable = true)
-- csMPa: string (nullable = true)

In [0]:
display(train_df.summary())
# display(train_df.describe())
# train_df.describe().show()

summary,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
count,740,740,740,740,740,740,740,740,740
mean,284.65527027026997,73.31797297297295,51.68783783783782,178.9208108108108,6.15513513513513,980.6385135135129,781.8905405405394,48.58108108108108,36.96363513513513
stddev,100.85775141703847,87.15508637558649,62.40102409880701,22.721371063280493,6.291589687322423,70.90189807541634,82.0426577530396,67.26497333330794,17.686493394911164
min,102.0,0.0,0.0,121.8,0.0,801.0,594.0,3,2.33
25%,203.5,0.0,0.0,160.6,0.0,936.2,746.6,7,23.52
50%,254.0,24.0,0.0,181.1,5.8,968.0,781.2,28,35.96
75%,362.6,133.0,118.3,192.0,10.4,1040.6,845.0,56,49.2
max,540.0,359.4,174.7,228.0,32.2,1145.0,992.6,365,82.6


In [0]:
display(test_df.summary())
# display(test_df.describe())
# test_df.describe().show()

summary,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
count,290,290,290,290,290,290,290,290,0
mean,272.26896551724155,75.3703448275862,60.56896551724137,188.32034482758624,6.331034482758619,953.2206896551722,752.3755172413795,38.213793103448275,null
stddev,112.9813283512792,84.13360821349904,67.59263490236042,15.485469242685207,5.08154973556983,90.1626069666602,71.04988880750282,50.60549448496449,null
min,132.0,0.0,0.0,127.0,0.0,801.0,612.0,1,null
25%,154.8,0.0,0.0,178.5,0.0,878.0,697.7,28,null
50%,287.3,0.0,0.0,189.0,7.0,949.4,763.0,28,null
75%,331.0,145.0,113.2,196.0,10.0,1002.0,806.0,28,null
max,540.0,260.0,200.1,247.0,22.1,1125.0,896.0,360,null


In [0]:
display(train_df)

cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.3
266.0,114.0,0.0,228.0,0.0,932.0,670.0,90,47.03
380.0,95.0,0.0,228.0,0.0,932.0,594.0,365,43.7
380.0,95.0,0.0,228.0,0.0,932.0,594.0,28,36.45
266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.85
475.0,0.0,0.0,228.0,0.0,932.0,594.0,28,39.29


In [0]:
display(test_df)

cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
297.0,0.0,0.0,186.0,0.0,1040.0,734.0,7,null
480.0,0.0,0.0,192.0,0.0,936.0,721.0,28,null
480.0,0.0,0.0,192.0,0.0,936.0,721.0,90,null
397.0,0.0,0.0,186.0,0.0,1040.0,734.0,28,null
281.0,0.0,0.0,186.0,0.0,1104.0,774.0,7,null
281.0,0.0,0.0,185.0,0.0,1104.0,774.0,28,null
500.0,0.0,0.0,200.0,0.0,1125.0,613.0,1,null
500.0,0.0,0.0,200.0,0.0,1125.0,613.0,3,null
500.0,0.0,0.0,200.0,0.0,1125.0,613.0,7,null
500.0,0.0,0.0,200.0,0.0,1125.0,613.0,14,null


### ML with SparkML

#### Select Model

**Steps:**

1. train validation split
1. build vector of features
1. transform
1. train

In [0]:
# ref:
# https://towardsdatascience.com/from-scikit-learn-to-spark-ml-f2886fb46852

from pyspark.sql.dataframe import DataFrame
from pyspark.ml.feature import VectorAssembler, StandardScaler, PolynomialExpansion
from pyspark.ml.regression import LinearRegression
from pyspark.ml.pipeline import Pipeline

def train_valid_split(df: DataFrame, target_col_name: str, test_size: float, seed=None):
  assert (0.0 < test_size < 1.0), "Invalid value for test_size"
  assembler = VectorAssembler().setInputCols(df.drop(target_col_name).columns).setOutputCol('features')
  train, test = df.randomSplit(weights=[1.0 - test_size, test_size], seed = seed)
  return (assembler.transform(train), assembler.transform(test))

test_size=0.15
features_list = train_df.drop('csMPa').columns
# traindf, validdf = train_valid_split(train_df, target_col_name='csMPa', test_size=test_size, seed=7)
traindf, validdf = train_df.randomSplit(weights=[1.0 -test_size, test_size], seed = 7)

# Transform & Train
model = Pipeline(stages=[
  VectorAssembler(inputCols=features_list, outputCol='features'),
  StandardScaler(inputCol='features', outputCol='scaledFeatures', withMean=False, withStd=True), 
  PolynomialExpansion(inputCol='scaledFeatures', outputCol='scaledPolyExpandedFeatures', degree=2),
  LinearRegression(labelCol='csMPa', featuresCol='scaledPolyExpandedFeatures', predictionCol='prediction')
])

predY = model.fit(traindf).transform(validdf)
display(predY)

# Evaluate
from pyspark.ml.evaluation import RegressionEvaluator

rmse = RegressionEvaluator().setLabelCol('csMPa').setPredictionCol('prediction').setMetricName('rmse')
r2 = RegressionEvaluator().setLabelCol('csMPa').setPredictionCol('prediction').setMetricName('r2')
print('Root Mean Squared Error =', rmse.evaluate(predY))
print('R^2 =', r2.evaluate(predY))

cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa,features,scaledFeatures,scaledPolyExpandedFeatures,prediction
108.3,162.4,0.0,203.5,0.0,938.2,849.0,7,7.72,"Map(vectorType -> dense, length -> 8, values -> List(108.3, 162.4, 0.0, 203.5, 0.0, 938.2, 849.0, 7.0))","Map(vectorType -> dense, length -> 8, values -> List(1.0776026669021281, 1.864053147711785, 0.0, 9.210019274527367, 0.0, 13.268806491002964, 10.578441151475918, 0.10319857898508855))","Map(vectorType -> dense, length -> 44, values -> List(1.0776026669021281, 1.1612275077145788, 1.864053147711785, 2.008708643221526, 3.4746941374942137, 0.0, 0.0, 0.0, 0.0, 9.210019274527367, 9.924741332450694, 17.167965419168947, 0.0, 84.8244550371656, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.268806491002964, 14.298501261313064, 24.73376050593264, 0.0, 122.20596353211114, 0.0, 176.0612256956824, 10.578441151475918, 11.399356396497668, 19.718776526292565, 0.0, 97.42764689954667, 0.0, 140.36328861539653, 111.90341719523914, 0.10319857898508855, 0.11120706393484134, 0.19236763599653758, 0.0, 0.9504609015565004, 0.0, 1.369321974699625, 1.0916800947096985, 0.010649946704541561))",13.338625966644031
108.3,162.4,0.0,203.5,0.0,938.2,849.0,90,29.23,"Map(vectorType -> dense, length -> 8, values -> List(108.3, 162.4, 0.0, 203.5, 0.0, 938.2, 849.0, 90.0))","Map(vectorType -> dense, length -> 8, values -> List(1.0776026669021281, 1.864053147711785, 0.0, 9.210019274527367, 0.0, 13.268806491002964, 10.578441151475918, 1.3268388726654243))","Map(vectorType -> dense, length -> 44, values -> List(1.0776026669021281, 1.1612275077145788, 1.864053147711785, 2.008708643221526, 3.4746941374942137, 0.0, 0.0, 0.0, 0.0, 9.210019274527367, 9.924741332450694, 17.167965419168947, 0.0, 84.8244550371656, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.268806491002964, 14.298501261313064, 24.73376050593264, 0.0, 122.20596353211114, 0.0, 176.0612256956824, 10.578441151475918, 11.399356396497668, 19.718776526292565, 0.0, 97.42764689954667, 0.0, 140.36328861539653, 111.90341719523914, 1.3268388726654243, 1.4298051077336744, 2.4732981770983407, 0.0, 12.220211591440721, 0.0, 17.605568246138038, 14.03588693198184, 1.760501394016054))",32.9795210519369
122.6,183.9,0.0,203.5,0.0,958.2,800.1,28,24.29,"Map(vectorType -> dense, length -> 8, values -> List(122.6, 183.9, 0.0, 203.5, 0.0, 958.2, 800.1, 28.0))","Map(vectorType -> dense, length -> 8, values -> List(1.219889999650978, 2.1108335829076186, 0.0, 9.210019274527367, 0.0, 13.55166316316248, 9.969152844871475, 0.4127943159403542))","Map(vectorType -> dense, length -> 44, values -> List(1.219889999650978, 1.488131611248463, 2.1108335829076186, 2.5749847787164475, 4.455618414730615, 0.0, 0.0, 0.0, 0.0, 9.210019274527367, 11.23521040958869, 19.440817983898828, 0.0, 84.8244550371656, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.55166316316248, 16.53153837138045, 28.60530570905545, 0.0, 124.81107893462895, 0.0, 183.64757448781492, 9.969152844871475, 12.16126986045081, 21.043222618093736, 0.0, 91.81608985197562, 0.0, 135.09860137578121, 99.38400844440903, 0.4127943159403542, 0.5035636579284044, 0.8713401049202774, 0.0, 3.8018436062260017, 0.0, 5.594049525291752, 4.115209629103557, 0.17039914727266497))",20.758544810342755
133.0,200.0,0.0,192.0,0.0,927.4,839.2,3,6.88,"Map(vectorType -> dense, length -> 8, values -> List(133.0, 200.0, 0.0, 192.0, 0.0, 927.4, 839.2, 3.0))","Map(vectorType -> dense, length -> 8, values -> List(1.3233716961955961, 2.29563195531008, 0.0, 8.689551354836631, 0.0, 13.116063888036823, 10.456334292483618, 0.04422796242218081))","Map(vectorType -> dense, length -> 44, values -> List(1.3233716961955961, 1.751312646291609, 2.29563195531008, 3.0379743545395135, 5.269926074240781, 0.0, 0.0, 0.0, 0.0, 8.689551354836631, 11.499506315628892, 19.94801176747097, 0.0, 75.50830274834313, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.116063888036823, 17.357427714921098, 30.109655389265903, 0.0, 113.97271072841419, 0.0, 172.03113191506364, 10.456334292483618, 13.837616848632225, 24.00389

Root Mean Squared Error = 7.576051707614912
R^2 = 0.7769443190499261

#### Predict

In [0]:
pred_df = model.fit(train_df)\
               .transform(test_df)
display(pred_df)

cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa,features,scaledFeatures,scaledPolyExpandedFeatures,prediction
297.0,0.0,0.0,186.0,0.0,1040.0,734.0,7,null,"Map(vectorType -> dense, length -> 8, values -> List(297.0, 0.0, 0.0, 186.0, 0.0, 1040.0, 734.0, 7.0))","Map(vectorType -> dense, length -> 8, values -> List(2.94474143858244, 0.0, 0.0, 8.186125717588869, 0.0, 14.668154566098943, 8.946565361271539, 0.1040660488381369))","Map(vectorType -> dense, length -> 44, values -> List(2.94474143858244, 8.671502140104579, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.186125717588869, 24.106023622029355, 0.0, 0.0, 67.01265426416987, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.668154566098943, 43.19392257832379, 0.0, 0.0, 120.07535732311115, 0.0, 215.15475837496928, 8.946565361271539, 26.34532175232258, 0.0, 0.0, 73.23770878799469, 0.0, 131.22960355483775, 80.04103176350374, 0.1040660488381369, 0.3064476063632057, 0.0, 0.0, 0.8518977587217317, 0.0, 1.5264568894409933, 0.9310337078196679, 0.010829742520781494))",12.537645467304174
480.0,0.0,0.0,192.0,0.0,936.0,721.0,28,null,"Map(vectorType -> dense, length -> 8, values -> List(480.0, 0.0, 0.0, 192.0, 0.0, 936.0, 721.0, 28.0))","Map(vectorType -> dense, length -> 8, values -> List(4.759178082557479, 0.0, 0.0, 8.450194289123994, 0.0, 13.201339109489048, 8.788111206371635, 0.4162641953525476))","Map(vectorType -> dense, length -> 44, values -> List(4.759178082557479, 22.649776021495487, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.450194289123994, 40.215979454151295, 0.0, 0.0, 71.40578352394375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.201339109489048, 62.82752375028915, 0.0, 0.0, 111.55388035179358, 0.0, 174.27535428372508, 8.788111206371635, 41.824186240441655, 0.0, 0.0, 74.26124712826817, 0.0, 116.01483616721285, 77.23089857555472, 0.4162641953525476, 1.9810754350752695, 0.0, 0.0, 3.517513326334892, 0.0, 5.4952448019875755, 3.6581760399889953, 0.1732758803325039))",38.147177814829774
480.0,0.0,0.0,192.0,0.0,936.0,721.0,90,null,"Map(vectorType -> dense, length -> 8, values -> List(480.0, 0.0, 0.0, 192.0, 0.0, 936.0, 721.0, 90.0))","Map(vectorType -> dense, length -> 8, values -> List(4.759178082557479, 0.0, 0.0, 8.450194289123994, 0.0, 13.201339109489048, 8.788111206371635, 1.3379920564903316))","Map(vectorType -> dense, length -> 44, values -> List(4.759178082557479, 22.649776021495487, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.450194289123994, 40.215979454151295, 0.0, 0.0, 71.40578352394375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.201339109489048, 62.82752375028915, 0.0, 0.0, 111.55388035179358, 0.0, 174.27535428372508, 8.788111206371635, 41.824186240441655, 0.0, 0.0, 74.26124712826817, 0.0, 116.01483616721285, 77.23089857555472, 1.3379920564903316, 6.367742469884795, 0.0, 0.0, 11.306292834647868, 0.0, 17.663286863531493, 11.758422985678912, 1.7902227432312268))",52.96760092270506
397.0,0.0,0.0,186.0,0.0,1040.0,734.0,28,null,"Map(vectorType -> dense, length -> 8, values -> List(397.0, 0.0, 0.0, 186.0, 0.0, 1040.0, 734.0, 28.0))","Map(vectorType -> dense, length -> 8, values -> List(3.936236872448582, 0.0, 0.0, 8.186125717588869, 0.0, 14.668154566098943, 8.946565361271539, 0.4162641953525476))","Map(vectorType -> dense, length -> 44, values -> List(3.936236872448582, 15.493960716023794, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.186125717588869, 32.22252989207291, 0.0, 0.0, 67.01265426416987, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.668154566098943, 57.73733085385369, 0.0, 0.0, 120.07535732311115, 0.0, 215.15475837496928, 8.946565361271539, 35.215800456808296, 0.0, 0.0, 73.23770878799469, 0.0, 131.22960355483775, 80.04103176350374, 0.4162641953525476, 1.6385144744268374, 0.0, 0.0, 3.4075910348869267, 0.0, 6.105827557763973, 3.7241348312786715, 0.1732758803325039))",36.15516925824613
281.0,0.0,0.0,186.0,0.0,1104.0,774.0,7,null,"Map(vectorType -> dense, length -> 8, values -> List(281.0, 0.0, 0.0, 186.0, 0.0, 1104.0, 774.0, 7.0))","Map(vectorType -> dense, length -> 8, values -> List(2.7861021691638577, 0.0, 0.0, 8.186

#### Save

In [0]:
output = pred_df.select(features_list + ['prediction']).withColumnRenamed('prediction', 'csMPa')
display(output)
output.coalesce(1).write.mode('overwrite').csv('/mnt/hackathon/Submission_Building_Strength_Test_900802.csv', header=True)

cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
297.0,0.0,0.0,186.0,0.0,1040.0,734.0,7,12.537645467304174
480.0,0.0,0.0,192.0,0.0,936.0,721.0,28,38.147177814829774
480.0,0.0,0.0,192.0,0.0,936.0,721.0,90,52.96760092270506
397.0,0.0,0.0,186.0,0.0,1040.0,734.0,28,36.15516925824613
281.0,0.0,0.0,186.0,0.0,1104.0,774.0,7,18.321940088916563
281.0,0.0,0.0,185.0,0.0,1104.0,774.0,28,23.64080616249339
500.0,0.0,0.0,200.0,0.0,1125.0,613.0,1,38.31155642522299
500.0,0.0,0.0,200.0,0.0,1125.0,613.0,3,38.91648846954922
500.0,0.0,0.0,200.0,0.0,1125.0,613.0,7,40.11177454710469
500.0,0.0,0.0,200.0,0.0,1125.0,613.0,14,42.15675406389437


### ML with Pandas

In [0]:
import pandas as pd

tr_df = train_df.toPandas()
ts_df = test_df.toPandas()

print('dataframe type is:', type(ts_df))
ts_df.head()

dataframe type is: <class 'pandas.core.frame.DataFrame'>
Out[11]:

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,297.0,0.0,0.0,186.0,0.0,1040.0,734.0,7,None
1,480.0,0.0,0.0,192.0,0.0,936.0,721.0,28,None
2,480.0,0.0,0.0,192.0,0.0,936.0,721.0,90,None
3,397.0,0.0,0.0,186.0,0.0,1040.0,734.0,28,None
4,281.0,0.0,0.0,186.0,0.0,1104.0,774.0,7,None


#### Select Model

In [0]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline


train_y = tr_df['csMPa']
train_X = tr_df.drop(columns='csMPa')
test_X = ts_df.drop(columns='csMPa')

trainX, validX, trainY, validY = train_test_split(train_X, train_y, test_size=0.15, random_state=7)

# Transform & Train
model = make_pipeline(StandardScaler(), PolynomialFeatures(2), LinearRegression())

model.fit(trainX, trainY)
predY = model.predict(validX)

# Evaluate
from sklearn.metrics import mean_squared_error, r2_score

rmse = mean_squared_error(y_pred=predY, y_true=validY) ** 0.5
r2 = r2_score(y_pred=predY, y_true=validY)
print('Root Mean Squared Error =', rmse)
print('R^2 =', r2)

Root Mean Squared Error = 7.3467939968537515
R^2 = 0.8321038505834102

#### Predict

In [0]:
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
pred_y

Out[13]: array([ 12.53764452, 38.14717789, 52.96760101, 36.15516943,
 18.32194016, 23.64080626, 38.31155632, 38.91648837,
 40.11177445, 42.15675398, 46.06813239, 59.5262805 ,
 61.7506266 , 66.02073816, 82.0690827 , 97.05560039,
 102.20196059, 24.09735736, 25.9301423 , 29.41713154,
 35.67678748, 41.99736295, 51.94952287, 21.78934619,
 22.3628979 , 23.4954233 , 25.43057165, 29.1222877 ,
 50.46694114, 43.48803082, 18.81123171, 18.94745533,
 28.95180262, 34.61892373, 24.23576734, 23.47824519,
 24.76170097, 30.20977216, 18.9998355 , 24.13932598,
 18.07549406, 19.26253439, 24.39529798, 36.42340287,
 14.11772644, 20.08009275, 25.70754519, 39.19616007,
 19.99153376, 25.86922549, 40.09664205, 52.43987719,
 47.605875 , 21.05533343, 46.13488733, 85.11764849,
 60.52780158, 73.20817763, 76.04839619, 58.22849924,
 14.89250498, 19.87530635, 29.6500984 , 22.35220682,
 29.01327234, 42.25630683, 29.00226244, 34.67463365,
 14.78024795, 19.87530635, 16.92258177, 22.52724259,
 35.94856787, 47.12167051, 39.94740333, 48.25130064,
 49.50477109, 51.65157328, 55.76659701, 71.12812401,
 89.26701549, 23.17170805, 26.61529197, 48.67642552,
 24.46306018, 36.20860631, 30.85006858, 49.35653216,
 57.44387226, 56.27409093, 82.87098881, 59.68015354,
 58.51020234, 60.20381247, 67.42721454, 52.14623972,
 58.55702384, 52.03674841, 57.01435299, 44.43580261,
 59.2934768 , 47.01431148, 30.51588164, 53.91555213,
 42.14840155, 49.42780557, 57.87348876, 54.731684 ,
 54.05509146, 33.08368342, 57.05636743, 66.21409345,
 38.16069493, 44.20807071, 49.17633196, 58.8525839 ,
 59.39567422, 38.57001679, 55.42935857, 54.32767481,
 44.73869346, 51.7082551 , 55.4778962 , 50.21710163,
 63.66127774, 47.86401427, 55.8158457 , 76.3009921 ,
 52.25447226, 57.60902846, 39.793664 , 59.15338394,
 58.43712231, 72.73724052, 32.81389127, 52.33592907,
 39.39235984, 76.72863419, 33.98448361, 55.56453363,
 29.49411581, 30.70958437, 42.87477205, 71.5617687 ,
 58.76721125, 46.6082098 , 44.61910092, 62.30345997,
 46.88785706, 39.87760618, 56.73797939, 24.10803225,
 56.38268545, 36.50985008, 29.85013228, 49.52072879,
 61.038253 , 49.26680905, 35.92586472, 37.83256828,
 54.63549641, 55.21867165, 61.51477567, 22.62704774,
 61.14903722, 32.18421099, 51.4201422 , 48.28996902,
 63.15217543, 29.80151273, 36.18586914, 63.41536469,
 39.44401274, 48.95691412, 52.08746509, 49.04840073,
 50.94435244, 26.05439826, 46.75074904, 49.37548858,
 45.9538119 , 43.49759533, 40.93904788, 62.76199223,
 47.88935509, 34.00728661, 61.00239989, 58.80788769,
 26.92863808, 16.66828143, 45.61947104, 8.34910353,
 48.98104728, 32.87990179, 25.95316908, 44.47956519,
 72.56199116, 32.97520292, 52.65086685, 41.21136141,
 74.67000132, 34.32216751, 55.4597025 , 29.2141342 ,
 30.86226728, 44.58356506, 70.29153169, 58.29537825,
 46.72256467, 44.99685544, 61.58422546, 46.94216938,
 39.91328728, 56.63034796, 22.88598986, 55.01503227,
 36.81507489, 31.22336888, 50.43182676, 60.10624779,
 49.72653905, 37.1682797 , 38.14993437, 54.32064052,
 56.97445377, 61.90905448, 23.07751424, 61.50199333,
 32.24194367, 52.45549464, 48.37347089, 62.98089914,
 31.28411051, 37.93127952, 63.44921863, 39.22862602,
 48.4281822 , 52.11114997, 49.62080009, 49.25413844,
 26.73541882, 45.79442976, 49.39017333, 47.07479739,
 44.00108659, 42.47184139, 56.2403718 , 82.17213321,
 60.0734484 , 57.58708613, 60.8735619 , 67.56812989,
 52.00782397, 59.20478225, 51.11585991, 57.00432475,
 45.67971879, 59.28694382, 46.42275462, 29.27046055,
 53.49466588, 42.34524 , 48.19083698, 56.71869071,
 56.04847915, 55.03141352, 31.87895169, 56.096438 ,
 67.44240169, 38.57768184, 42.98332189, 49.59222359,
 58.64801746, 59.46068914, 39.8208852 , 56.79689926,
 53.01376445, 44.93576095, 53.15707589, 54.39165793,
 50.11223993, 64.00668185, 47.62938056, 56.31904386,
 75.4031945 , 51.9181117 , 58.93626149, 40.06110592,
 60.30534912, 56.74366416])

#### Save

In [0]:
test_X['csMPa'] = pred_y
test_X.to_csv('Submission_Building_Strength_Test_900802.csv', header=True)
test_X.head(10)

Out[14]:

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,297.0,0.0,0.0,186.0,0.0,1040.0,734.0,7,12.537645
1,480.0,0.0,0.0,192.0,0.0,936.0,721.0,28,38.147178
2,480.0,0.0,0.0,192.0,0.0,936.0,721.0,90,52.967601
3,397.0,0.0,0.0,186.0,0.0,1040.0,734.0,28,36.155169
4,281.0,0.0,0.0,186.0,0.0,1104.0,774.0,7,18.321940
5,281.0,0.0,0.0,185.0,0.0,1104.0,774.0,28,23.640806
6,500.0,0.0,0.0,200.0,0.0,1125.0,613.0,1,38.311556
7,500.0,0.0,0.0,200.0,0.0,1125.0,613.0,3,38.916488
8,500.0,0.0,0.0,200.0,0.0,1125.0,613.0,7,40.111774
9,500.0,0.0,0.0,200.0,0.0,1125.0,613.0,14,42.156754
